<a href="https://colab.research.google.com/github/shruteeegrg/Machine-Learning-with-Python/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.7/633.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.8 MB/s eta 0:00:00
2.19.0-dev20250110


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-14 17:46:54--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2025-01-14 17:46:54 (9.99 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-14 17:46:54--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [33]:
train_dataset = pd.read_csv('train-data.tsv',sep='\t',header=None,names=['class', 'text'],encoding='utf-8')
valid_dataset = pd.read_csv('valid-data.tsv',sep='\t',header=None,names=['class', 'text'],encoding='utf-8')

In [4]:
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])
test_df.dropna()
train_df.head()

,type,msg
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [34]:
VOCAB_SIZE = 88584
BATCH_SIZE = 64

In [35]:
# Preprocessing
def preprocess(df):
    data = df.copy()
    data.loc[data['class'] == 'ham','class'] = 0
    data.loc[data['class'] == 'spam','class'] = 1
    labels = data.pop('class').astype(np.float32)
    new_data = []
    for i in data['text']:
      new_data.append(i)
    return new_data, labels

train_data, train_labels = preprocess(train_dataset)
valid_data, valid_labels = preprocess(valid_dataset)

# Calculating the maximum message length
MAXLEN_train = max(train_data, key=len)
MAXLEN_valid = max(valid_data, key=len)
MAXLEN = max(len(MAXLEN_train.strip()), len(MAXLEN_valid.strip()))

In [36]:
dataset = tf.keras.datasets.imdb

In [8]:
test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["msg"].values, test_labels)
)
test_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [37]:
from keras.preprocessing.sequence import pad_sequences

In [38]:
# Taking words indexes from preproc lib
word_index = dataset.get_word_index()

# Function for encoding text message
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return pad_sequences([tokens], MAXLEN)[0]

# Func for prep text message for model
def prep_to_mod(data_for_prep):
    new_data = []
    for i in data_for_prep:
        new_data.append(encode_text(i))
    return np.array(new_data)

# Prep data for model
new_train_data = prep_to_mod(train_data)
new_valid_data = prep_to_mod(valid_data)

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [39]:
# Creating model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [40]:
# Fitting model
model.fit(new_train_data, train_labels, epochs=5, validation_split = 0.2)

Epoch 1/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 55s 507ms/step - acc: 0.8498 - loss: 0.4145 - val_acc: 0.9414 - val_loss: 0.1664
Epoch 2/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 49s 464ms/step - acc: 0.9579 - loss: 0.1439 - val_acc: 0.9737 - val_loss: 0.1000
Epoch 3/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 87s 516ms/step - acc: 0.9712 - loss: 0.1083 - val_acc: 0.9725 - val_loss: 0.1084
Epoch 4/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 79s 486ms/step - acc: 0.9721 - loss: 0.1048 - val_acc: 0.9809 - val_loss: 0.0854
Epoch 5/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 82s 486ms/step - acc: 0.9799 - loss: 0.0765 - val_acc: 0.9833 - val_loss: 0.0587


In [41]:
results = model.evaluate(new_valid_data, valid_labels)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - acc: 0.9799 - loss: 0.0553
[0.06250772625207901, 0.9791666865348816]


In [42]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    encoded_text = encode_text(pred_text)
    pred = np.zeros((1, MAXLEN))
    pred[0] = encoded_text
    result = model.predict(pred).flatten()[0]
    label = ""
    if result > 0.5:
        label = "spam"
    else:
        label = "ham"
    prediction = [result, label]
    return (prediction)

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
[0.6429214, 'spam']


In [43]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
You passed the challenge. Great job!
